Bhashini

In [52]:
from bhashini_translator import Bhashini
from dotenv import load_dotenv

In [53]:
load_dotenv()

True

In [54]:
def bhashini_translate(sourceLanguage, targetLanguage, text):
    bhashini = Bhashini(sourceLanguage, targetLanguage)
    return bhashini.translate(text)

In [70]:
print(bhashini_translate('en','ta', 'How are you?'))

எப்படி இருக்கீங்க?


In [56]:
prompt ="You are an Educational Assistant. The user may ask about various educational topics, technical queries, and other concerns. Respond in a detailed but simple manner. for example the user ask a query like of  this 'What is the boiling point of water?' you should respond in the form of like this 'The boiling point of water is 100°C (212°F) at sea level. This is the temperature at which water changes from a liquid to a gas'."

In [107]:
from openai import OpenAI

def LLM(query, content):
# Point to the local server
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

    passLLM = f'Question:{query}\nContent:{content}'
    #print(passLLM)

    completion = client.chat.completions.create(
    model="TheBloke/Llama-2-7B-Chat-GGUF",
    messages=[
        {"role": "system", "content": "You are an Educational Assistant. The user may ask about various educational topics, technical queries, and other concerns. Respond in a detailed but simple manner. for example the user ask a query like of  this 'What is the boiling point of water?' you should respond in the form of like this 'The boiling point of water is 100°C (212°F) at sea level. This is the temperature at which water changes from a liquid to a gas'."},
        {"role": "user", "content": passLLM}
    ],
    temperature=1,
    )

    print(completion.choices[0].message)
    return completion.choices[0].message

In [58]:
import fitz  # PyMuPDF

# Open the PDF file
doc = fitz.open("D:/Samaaja/Vil-Bert.pdf")

# Extract text from each page
pdf_text = ""
for page in doc:
    pdf_text += page.get_text()

len(pdf_text)

45273

In [59]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_Splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=200)

In [ ]:
text_Chunks = text_Splitter.create_documents([pdf_text])
text_Chunks

In [ ]:
text = [text_Chunk.page_content for text_Chunk in text_Chunks]
text

In [ ]:
#without Built-in
def recursive_text_splitter(text, max_chunk_size=1000, overlap_size=200):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) >= max_chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap_size:]  # retain overlap words

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

chunks = recursive_text_splitter(pdf_text)
chunks

In [ ]:
import chromadb
import spacy
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="multilingualedtechrag")
#nlp = spacy.load('en_core_web_md')

In [ ]:
collection.add(documents=text, ids = [str(i) for i in range(len(text))])

In [95]:
question = "Is VilBERT is Parallel or Single Stream? "

In [96]:
results = collection.query(query_texts=[question], n_results=4)
answer = results['documents'][0]

In [97]:
answer

['network depth for each modality and enables cross-modal connections at different depths.\nOur model which we call ViLBERT is shown in Fig. 1 and consists of two parallel BERT-style\nmodels operating over image regions and text segments. Each stream is a series of transformer\nblocks (TRM) and novel co-attentional transformer layers (Co-TRM) which we introduce to enable\ninformation exchange between modalities. Given an image I represented as a set of region features\nv1, . . . , vT and a text input w0, . . . , wT , our model outputs ﬁnal representations hv0, . . . , hvT and\nhw0, . . . , hwT . Notice that exchange between the two streams is restricted to be between speciﬁc\n1Concurrent work [29] modelling language and video sequences takes this approach. See Sec. 5.\n3\nVision               &        Language BERT\n…\nℎ"0\nℎ$%\nℎ$&\nℎ$\'\nℎ$𝒯\n<IMG>\nℎ)0\nℎ*%\nℎ*&\nℎ*\'\nℎ*+\n<CLS>\nMan\nshopping\nfor\n<SEP>\n…\n…\n…\n<MASK>\n<MASK>\n<MASK>\n<MASK>\nMan shopping\n(a) Masked multi-moda

In [114]:
import re

def clean_and_extract_words(text):
    words = re.findall(r'[A-Za-z]+', text)
    return ' '.join(words)

def clean_segments_and_print(answer):
    cleaned_content = []

    for segment in answer:
        cleaned_segment = clean_and_extract_words(segment)
        cleaned_content.append(cleaned_segment)
    seg = ""
    for i, segment in enumerate(cleaned_content, start=1):
        #print(f"Segment {i}:")
        seg += segment
        #print(segment)
        #print("\n")
    return seg

cleaned_text = clean_segments_and_print(answer)


'ViLBERTisaparallelstreammodelmeaningitconsistsoftwoseparatestreamsofBERTstylemodelsoperatinginparallelEachstreamhasitsownseriesoftransformerblocksandcoattentionaltransformerlayerswhichenableinformationexchangebetweenmodalitiesTheinputimageisrepresentedasasetofregionfeaturesandtheinputtextisrepresentedasasequenceofwordsThefinalrepresentationsfrombothstreamsarecombinedandthemodeloutputsanalrepresentationTheViLBERTmodellearnsimportantvisuallinguisticrelationshipsduringpretrainingasdemonstratedbyitsperformanceonzeroshotcaptionbasedimageretrievaltasksThemodelsabilitytotransferknowledgetodownstreamtasksisevidentinitsperformanceacrossvarioustasksincludingVQAImageRetrievalandRefCOCOTheeffectofvisualstreamdepthisalsostudiedanditisfoundthatbothVQAandImageRetrievaltasksbenetfromgreaterdepthwhileVCRandRefCOCOseemtobenetfromshallowermodelsFinallytheimpactofthesizeofthepretrainingdatasetisexploredanditisfoundthatlargertrainingsetsresultinbetterperformance'

In [103]:
response = LLM(question, answer)

ChatCompletionMessage(content="ViLBERT is a parallel stream model, meaning it consists of two separate streams of BERT-style models operating in parallel. Each stream has its own series of transformer blocks and co-attentional transformer layers, which enable information exchange between modalities. The input image is represented as a set of region features, and the input text is represented as a sequence of words. The final representations from both streams are combined, and the model outputs a ﬁnal representation.\nThe ViLBERT model learns important visual-linguistic relationships during pretraining, as demonstrated by its performance on zero-shot caption-based image retrieval tasks. The model's ability to transfer knowledge to downstream tasks is evident in its performance across various tasks, including VQA, Image Retrieval, and RefCOCO+. The effect of visual stream depth is also studied, and it is found that both VQA and Image Retrieval tasks beneﬁt from greater depth, while VCR a

In [110]:
res_llm ="ViLBERT is a parallel stream model, meaning it consists of two separate streams of BERT-style models operating in parallel. Each stream has its own series of transformer blocks and co-attentional transformer layers, which enable information exchange between modalities. The input image is represented as a set of region features, and the input text is represented as a sequence of words. The final representations from both streams are combined, and the model outputs a ﬁnal representation.\nThe ViLBERT model learns important visual-linguistic relationships during pretraining, as demonstrated by its performance on zero-shot caption-based image retrieval tasks. The model's ability to transfer knowledge to downstream tasks is evident in its performance across various tasks, including VQA, Image Retrieval, and RefCOCO+. The effect of visual stream depth is also studied, and it is found that both VQA and Image Retrieval tasks beneﬁt from greater depth, while VCR and RefCOCO+ seem to beneﬁt from shallower models. Finally, the impact of the size of the pretraining dataset is explored, and it is found that larger training sets result in better performance"

In [115]:
res_llm

"ViLBERT is a parallel stream model, meaning it consists of two separate streams of BERT-style models operating in parallel. Each stream has its own series of transformer blocks and co-attentional transformer layers, which enable information exchange between modalities. The input image is represented as a set of region features, and the input text is represented as a sequence of words. The final representations from both streams are combined, and the model outputs a ﬁnal representation.\nThe ViLBERT model learns important visual-linguistic relationships during pretraining, as demonstrated by its performance on zero-shot caption-based image retrieval tasks. The model's ability to transfer knowledge to downstream tasks is evident in its performance across various tasks, including VQA, Image Retrieval, and RefCOCO+. The effect of visual stream depth is also studied, and it is found that both VQA and Image Retrieval tasks beneﬁt from greater depth, while VCR and RefCOCO+ seem to beneﬁt fro

In [116]:
translation_response = bhashini_translate('en','ta', res_llm)
translation_response

'VILBERT என்பது ஒரு இணையான ஸ்ட்ரீம் மாதிரியாகும், அதாவது இது இணையாக செயல்படும் BERT-பாணி மாதிரிகளின் இரண்டு தனித்தனி ஸ்ட்ரீம்களைக் கொண்டுள்ளது. ஒவ்வொரு ஸ்ட்ரீம் அதன் சொந்த தொடர் மின்மாற்றி தொகுதிகள் மற்றும் இணை கவனம் மின்மாற்றி அடுக்குகளைக் கொண்டுள்ளது, இது முறைகளுக்கு இடையில் தகவல் பரிமாற்றத்தை செயல்படுத்துகிறது. உள்ளீட்டு படம் பிராந்திய அம்சங்களின் தொகுப்பாக குறிப்பிடப்படுகிறது, மேலும் உள்ளீட்டு உரை சொற்களின் வரிசையாக குறிப்பிடப்படுகிறது. இரண்டு நீரோடைகளிலிருந்தும் இறுதி பிரதிநிதித்துவங்கள் இணைக்கப்படுகின்றன, மேலும் மாதிரி ஒரு இறுதி பிரதிநிதித்துவத்தை வெளியிடுகிறது. VILBERT மாதிரி முன் பயிற்சியின் போது முக்கியமான காட்சி-மொழியியல் உறவுகளைக் கற்றுக்கொள்கிறது, இது பூஜ்ஜிய-ஷாட் தலைப்பு-அடிப்படையிலான பட மீட்டெடுப்பு பணிகளில் அதன் செயல்திறனால் நிரூபிக்கப்பட்டுள்ளது. கீழ்நிலைப் பணிகளுக்கு அறிவை மாற்றும் மாதிரியின் திறன் VQA, பட மீட்டெடுப்பு மற்றும் ரெஃப்கோகோ + உள்ளிட்ட பல்வேறு பணிகளில் அதன் செயல்திறனில் தெளிவாகத் தெரிகிறது. காட்சி ஸ்ட்ரீம் ஆழத்தின் விளைவும் ஆய்வு செய்யப்படுகிறது, மேலும் வி.

In [86]:
type(translation_response)

str